In [222]:
import pandas as pd

In [223]:
#https://cdn.buenosaires.gob.ar/datosabiertos/datasets/agencia-de-proteccion-ambiental/calidad-aire/calidad-aire.csv
input_file_name = 'contaminantes.csv'

In [224]:
output_file_name = "contaminantes.xlsx"
output_sheet_name = "contaminantes"

In [225]:
df_contaminantes = pd.read_csv(input_file_name)

In [226]:
df_contaminantes.head()

,FECHA,HORA,CO_CENTENARIO,NO2_CENTENARIO,PM10_CENTENARIO,CO_CORDOBA,NO2_CORDOBA,PM10_CORDOBA,CO_LA_BOCA,NO2_LA_BOCA,PM10_LA_BOCA,CO_PALERMO,NO2_PALERMO,PM10_PALERMO
0,05NOV2010:00:00:00,11,s/d,22,NaN,0.64,s/d,NaN,0.22,18,NaN,NaN,NaN,NaN
1,01SEP2010:00:00:00,15,s/d,20,NaN,1.65,s/d,NaN,<0.05,12,NaN,NaN,NaN,NaN
2,01SEP2010:00:00:00,17,s/d,19,NaN,1.99,s/d,NaN,<0.05,12,NaN,NaN,NaN,NaN
3,01SEP2010:00:00:00,24,s/d,16,NaN,1.65,s/d,NaN,<0.05,12,NaN,NaN,NaN,NaN
4,02SEP2010:00:00:00,5,s/d,12,NaN,0.41,s/d,NaN,<0.05,7,NaN,NaN,NaN,NaN


In [227]:
df_contaminantes.shape

(104153, 14)

In [228]:
df_contaminantes['Año'] = [ i[5:9] for i in df_contaminantes['FECHA'] ]

In [229]:
df_contaminantes = df_contaminantes.loc[ df_contaminantes.Año == '2020' ]

In [230]:
df_contaminantes['Mes'] = [ i[2:5] for i in df_contaminantes['FECHA'] ]

In [231]:
MESES = { 'JAN' : '1', 'FEB' : '2', 'MAR' : '3', 'APR' : '4', 'MAY' : '5', 'JUN' : '6', 'JUL' : '7', 'AUG' : '8', 'SEP' : '9', 'OCT' : '10', 'NOV' : '11', 'DEC' : '12'}

In [232]:
df_contaminantes["Mes"] = df_contaminantes["Mes"].map(MESES).astype(int)

In [233]:
df_contaminantes['Tipo'] = 'Contaminantes'

In [234]:
df_contaminantes = df_contaminantes.dropna(subset=['PM10_CENTENARIO'])

In [235]:
df_contaminantes['PM10_CENTENARIO'] = df_contaminantes['PM10_CENTENARIO'].astype(int)

In [236]:
df = df_contaminantes.groupby(['Año','Mes','Tipo'],as_index=False).mean()[['Año','Mes','Tipo','PM10_CENTENARIO']].rename({'PM10_CENTENARIO':'Total'},axis=1)
df['Total'] = round(df['Total'],2)
df

,Año,Mes,Tipo,Total
0,2020,1,Contaminantes,20.65
1,2020,2,Contaminantes,18.40
2,2020,3,Contaminantes,17.53
3,2020,4,Contaminantes,14.09
4,2020,5,Contaminantes,20.00
5,2020,6,Contaminantes,17.54
6,2020,7,Contaminantes,19.44
7,2020,8,Contaminantes,26.99
8,2020,9,Contaminantes,20.17
9,2020,10,Contaminantes,18.97


In [237]:
df['Intervalo']=pd.cut(df['Total'],10,labels=[1,2,3,4,5,6,7,8,9,10])
df

,Año,Mes,Tipo,Total,Intervalo
0,2020,1,Contaminantes,20.65,6
1,2020,2,Contaminantes,18.40,4
2,2020,3,Contaminantes,17.53,3
3,2020,4,Contaminantes,14.09,1
4,2020,5,Contaminantes,20.00,5
5,2020,6,Contaminantes,17.54,3
6,2020,7,Contaminantes,19.44,5
7,2020,8,Contaminantes,26.99,10
8,2020,9,Contaminantes,20.17,5
9,2020,10,Contaminantes,18.97,4


In [238]:
enero = df.loc[ df.Mes == 1, ['Intervalo']].iloc[0,0]
diciembre = df.loc[ df.Mes == 12, ['Intervalo']].iloc[0,0]
diciembre_enero = int(((diciembre - enero)/enero)*100)
if diciembre_enero > 0:
    diciembre_enero = '+' + str(diciembre_enero) + ' %'
else:
    diciembre_enero = str(diciembre_enero) + ' %'

In [239]:
df['Diciembre Vs Enero'] = diciembre_enero
df

,Año,Mes,Tipo,Total,Intervalo,Diciembre Vs Enero
0,2020,1,Contaminantes,20.65,6,+33 %
1,2020,2,Contaminantes,18.40,4,+33 %
2,2020,3,Contaminantes,17.53,3,+33 %
3,2020,4,Contaminantes,14.09,1,+33 %
4,2020,5,Contaminantes,20.00,5,+33 %
5,2020,6,Contaminantes,17.54,3,+33 %
6,2020,7,Contaminantes,19.44,5,+33 %
7,2020,8,Contaminantes,26.99,10,+33 %
8,2020,9,Contaminantes,20.17,5,+33 %
9,2020,10,Contaminantes,18.97,4,+33 %


In [240]:
writer = pd.ExcelWriter(output_file_name, engine='xlsxwriter')

df.to_excel(writer, index=False, sheet_name=output_sheet_name)

workbook  = writer.book
worksheet = writer.sheets[output_sheet_name]

format = workbook.add_format()
format.set_align('center')
format.set_align('vcenter')

worksheet.set_column('A:F', 30, format)

writer.save()